# Рубежный контроль №2
## Трофимова Юлия РТ5-61 Вариант 15

Набор данных для выполнения задания: https://www.kaggle.com/noriuk/us-education-datasets-unification-project <br>
Будем кластеризовать данных с помощью алгоритмов MeanShift и DBSCAN

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
sns.set(style="ticks")

Загрузим данные и посмотрим на них

In [2]:
data = pd.read_csv('states_all_extended.csv')
data.shape

(1715, 266)

In [3]:
data.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,G08_HI_A_READING,G08_HI_A_MATHEMATICS,G08_AS_A_READING,G08_AS_A_MATHEMATICS,G08_AM_A_READING,G08_AM_A_MATHEMATICS,G08_HP_A_READING,G08_HP_A_MATHEMATICS,G08_TR_A_READING,G08_TR_A_MATHEMATICS
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Удаляем данные о средних оценках учеников разных классов по разным предметам, также удаляем все данные о поле учеников и их принадлежности к какой-либо этнической группе. Делаем это, как и последующие действия, чтобы упростить датасет и облегчить понимание. 

In [4]:
data = data.iloc[:,0:25]
data.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,G02_A_A,G03_A_A,G04_A_A,G05_A_A,G06_A_A,G07_A_A,G08_A_A,G09_A_A,G10_A_A,G11_A_A
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,56139.0,59481.0,57948.0,58970.0,59929.0,62904.0,58025.0,60590.0,50629.0,44000.0
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,10585.0,10135.0,9748.0,9896.0,9542.0,9392.0,8789.0,9187.0,7889.0,7057.0
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,56814.0,55630.0,55433.0,54630.0,53832.0,52793.0,49081.0,51426.0,46790.0,39584.0
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34517.0,34044.0,34632.0,34740.0,35017.0,37448.0,36011.0,33518.0,32700.0,29244.0
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,436488.0,424961.0,418418.0,410701.0,399776.0,388439.0,363296.0,396482.0,375327.0,329527.0


Удалим столбец PRIMARY_KEY, созданный из столбцов STATE и YEAR

In [5]:
data.drop(columns='PRIMARY_KEY', inplace = True)
data.head()

,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,...,G02_A_A,G03_A_A,G04_A_A,G05_A_A,G06_A_A,G07_A_A,G08_A_A,G09_A_A,G10_A_A,G11_A_A
0,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,735036.0,...,56139.0,59481.0,57948.0,58970.0,59929.0,62904.0,58025.0,60590.0,50629.0,44000.0
1,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,350902.0,...,10585.0,10135.0,9748.0,9896.0,9542.0,9392.0,8789.0,9187.0,7889.0,7057.0
2,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,1007732.0,...,56814.0,55630.0,55433.0,54630.0,53832.0,52793.0,49081.0,51426.0,46790.0,39584.0
3,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,483488.0,...,34517.0,34044.0,34632.0,34740.0,35017.0,37448.0,36011.0,33518.0,32700.0,29244.0
4,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,8520926.0,...,436488.0,424961.0,418418.0,410701.0,399776.0,388439.0,363296.0,396482.0,375327.0,329527.0


In [6]:
data.isnull().sum()

STATE                             0
YEAR                              0
ENROLL                          491
TOTAL_REVENUE                   440
FEDERAL_REVENUE                 440
STATE_REVENUE                   440
LOCAL_REVENUE                   440
TOTAL_EXPENDITURE               440
INSTRUCTION_EXPENDITURE         440
SUPPORT_SERVICES_EXPENDITURE    440
OTHER_EXPENDITURE               491
CAPITAL_OUTLAY_EXPENDITURE      440
A_A_A                            83
G01_A_A                          83
G02_A_A                          83
G03_A_A                          83
G04_A_A                          83
G05_A_A                          83
G06_A_A                          83
G07_A_A                          83
G08_A_A                          83
G09_A_A                          83
G10_A_A                          83
G11_A_A                          83
dtype: int64

Удалим столбец ENROLL, который должен совпадать со столбцом A_A_A и столбцы FEDERAL_REVENUE, STATE_REVENUE, LOCAL_REVENUE, оставив только TOTAL_REVENUE, также поступим с INSTRUCTION_EXPENDITURE, SUPPORT_SERVICES_EXPENDITURE, OTHER_EXPENDITURE,
CAPITAL_OUTLAY_EXPENDITURE.

In [7]:
data.drop(columns=['ENROLL',
                   'FEDERAL_REVENUE', 
                   'STATE_REVENUE', 
                   'LOCAL_REVENUE', 
                   'INSTRUCTION_EXPENDITURE',
                   'SUPPORT_SERVICES_EXPENDITURE',
                   'OTHER_EXPENDITURE',
                   'CAPITAL_OUTLAY_EXPENDITURE'], inplace = True)
data.head()

,STATE,YEAR,TOTAL_REVENUE,TOTAL_EXPENDITURE,A_A_A,G01_A_A,G02_A_A,G03_A_A,G04_A_A,G05_A_A,G06_A_A,G07_A_A,G08_A_A,G09_A_A,G10_A_A,G11_A_A
0,ALABAMA,1992,2678885.0,2653798.0,731634.0,58168.0,56139.0,59481.0,57948.0,58970.0,59929.0,62904.0,58025.0,60590.0,50629.0,44000.0
1,ALASKA,1992,1049591.0,972488.0,122487.0,11030.0,10585.0,10135.0,9748.0,9896.0,9542.0,9392.0,8789.0,9187.0,7889.0,7057.0
2,ARIZONA,1992,3258079.0,3401580.0,673477.0,58914.0,56814.0,55630.0,55433.0,54630.0,53832.0,52793.0,49081.0,51426.0,46790.0,39584.0
3,ARKANSAS,1992,1711959.0,1743022.0,441490.0,34929.0,34517.0,34044.0,34632.0,34740.0,35017.0,37448.0,36011.0,33518.0,32700.0,29244.0
4,CALIFORNIA,1992,26260025.0,27138832.0,5254844.0,443955.0,436488.0,424961.0,418418.0,410701.0,399776.0,388439.0,363296.0,396482.0,375327.0,329527.0


Поясним имеющиеся у нас на данный момент данные:<br>
STATE - штат, в котором проходило исследование, YEAR - год, G01_A_A-G11_A_A - количество учеников в соответствующих классах с 1-го по 11, A_A_A - общее количество учеников, TOTAL_REVENUE - общее количество доходов для штата, TOTAL_EXPENDITURE - общее количество расходов для штата

In [8]:
data.isnull().sum()

STATE                  0
YEAR                   0
TOTAL_REVENUE        440
TOTAL_EXPENDITURE    440
A_A_A                 83
G01_A_A               83
G02_A_A               83
G03_A_A               83
G04_A_A               83
G05_A_A               83
G06_A_A               83
G07_A_A               83
G08_A_A               83
G09_A_A               83
G10_A_A               83
G11_A_A               83
dtype: int64

Удалим строки с пустыми значениями в количестве учеников и заполним остальные пропуски средним значением

In [9]:
data.dropna(subset=['A_A_A'], inplace=True)
data.isnull().sum()

STATE                  0
YEAR                   0
TOTAL_REVENUE        357
TOTAL_EXPENDITURE    357
A_A_A                  0
G01_A_A                0
G02_A_A                0
G03_A_A                0
G04_A_A                0
G05_A_A                0
G06_A_A                0
G07_A_A                0
G08_A_A                0
G09_A_A                0
G10_A_A                0
G11_A_A                0
dtype: int64

In [10]:
from sklearn.impute import SimpleImputer
from sklearn.impute import MissingIndicator

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
data_rev = imp.fit_transform(data[['TOTAL_REVENUE']])
data_exp = imp.fit_transform(data[['TOTAL_EXPENDITURE']])

data['TOTAL_REVENUE'] = data_rev
data['TOTAL_EXPENDITURE'] = data_exp
data.isnull().sum()

STATE                0
YEAR                 0
TOTAL_REVENUE        0
TOTAL_EXPENDITURE    0
A_A_A                0
G01_A_A              0
G02_A_A              0
G03_A_A              0
G04_A_A              0
G05_A_A              0
G06_A_A              0
G07_A_A              0
G08_A_A              0
G09_A_A              0
G10_A_A              0
G11_A_A              0
dtype: int64

Отмасштабируем данные

In [11]:
sc_data = StandardScaler().fit_transform(data[data.columns[2:]])
for i in range(len(data.columns[2:])):
    data[data.columns[2:][i]] = sc_data[:,i]
data.head()

,STATE,YEAR,TOTAL_REVENUE,TOTAL_EXPENDITURE,A_A_A,G01_A_A,G02_A_A,G03_A_A,G04_A_A,G05_A_A,G06_A_A,G07_A_A,G08_A_A,G09_A_A,G10_A_A,G11_A_A
0,ALABAMA,1992,-0.618203,-0.618384,-0.172737,-0.163345,-0.169907,-0.126952,-0.140348,-0.123039,-0.111279,-0.077790,-0.129347,-0.163637,-0.222516,-0.254038
1,ALASKA,1992,-0.775015,-0.777057,-0.749816,-0.732384,-0.730525,-0.737622,-0.743349,-0.741097,-0.749084,-0.755200,-0.761048,-0.753816,-0.761965,-0.763070
2,ARIZONA,1992,-0.562458,-0.547813,-0.227832,-0.154339,-0.161600,-0.174609,-0.171812,-0.177699,-0.188456,-0.205785,-0.244099,-0.268853,-0.270970,-0.314885
3,ARKANSAS,1992,-0.711265,-0.704338,-0.447606,-0.443881,-0.436002,-0.441742,-0.432041,-0.428201,-0.426618,-0.400038,-0.411788,-0.474462,-0.448809,-0.457359
4,CALIFORNIA,1992,1.651385,1.692381,4.112353,4.493789,4.510925,4.395964,4.369276,4.306800,4.190547,4.043170,3.787299,3.692878,3.875709,3.680192


In [12]:
data = pd.get_dummies(data, columns=['STATE'])
data.head()

,YEAR,TOTAL_REVENUE,TOTAL_EXPENDITURE,A_A_A,G01_A_A,G02_A_A,G03_A_A,G04_A_A,G05_A_A,G06_A_A,...,STATE_SOUTH_DAKOTA,STATE_TENNESSEE,STATE_TEXAS,STATE_UTAH,STATE_VERMONT,STATE_VIRGINIA,STATE_WASHINGTON,STATE_WEST_VIRGINIA,STATE_WISCONSIN,STATE_WYOMING
0,1992,-0.618203,-0.618384,-0.172737,-0.163345,-0.169907,-0.126952,-0.140348,-0.123039,-0.111279,...,0,0,0,0,0,0,0,0,0,0
1,1992,-0.775015,-0.777057,-0.749816,-0.732384,-0.730525,-0.737622,-0.743349,-0.741097,-0.749084,...,0,0,0,0,0,0,0,0,0,0
2,1992,-0.562458,-0.547813,-0.227832,-0.154339,-0.161600,-0.174609,-0.171812,-0.177699,-0.188456,...,0,0,0,0,0,0,0,0,0,0
3,1992,-0.711265,-0.704338,-0.447606,-0.443881,-0.436002,-0.441742,-0.432041,-0.428201,-0.426618,...,0,0,0,0,0,0,0,0,0,0
4,1992,1.651385,1.692381,4.112353,4.493789,4.510925,4.395964,4.369276,4.306800,4.190547,...,0,0,0,0,0,0,0,0,0,0


Кластеризуем данные с помощью алгоритма MeanShift

In [13]:
from sklearn.cluster import MeanShift
from sklearn.metrics import silhouette_score

ms = MeanShift()
pred = ms.fit_predict(data)
print(np.unique(pred))

print('Silhouette: ', silhouette_score(data, pred))

[0 1 2]
Silhouette:  0.5379291670800983


Кластеризуем данные с помощью алгоритма DBSCAN

In [14]:
from sklearn.cluster import DBSCAN

dbs = DBSCAN(eps=3, min_samples=3)
pred = dbs.fit_predict(data)

print(np.unique(pred))
print('Silhouette: ', silhouette_score(data, pred))

[-1  0  1]
Silhouette:  0.38654704431850767


Так как в нашем наборе данных мы не знаем истинных значений меток кластеров, то для оценки качества моделей мы можем использовать только метрику Коэффициент силуэта. Сравнивая метрики двух полученных моделей, можно сделать вывод, что первая модель(MeanShift) лучше.